In [1]:
import dgl.function as fn
from dgl.nn import GATConv
import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


In [2]:
from ogb.nodeproppred import DglNodePropPredDataset, NodePropPredDataset

dataset = DglNodePropPredDataset(name = 'ogbn-mag')

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
graph, label = dataset[0] # graph: dgl graph object, label: torch tensor of shape (num_nodes, num_tasks)

Downloaded 0.40 GB: 100%|████████████████████████████████████████████████████████████| 413/413 [00:32<00:00, 12.78it/s]


Extracting dataset\mag.zip
Loading necessary files...
This might take a while.


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Processing graphs...
Converting graphs into DGL objects...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.80s/it]


Saving...


In [4]:
print(vars(dataset))

{'name': 'ogbn-mag', 'dir_name': 'ogbn_mag', 'original_root': 'dataset', 'root': 'dataset\\ogbn_mag', 'meta_info': num tasks                                                                1
num classes                                                            349
eval metric                                                            acc
task type                                        multiclass classification
download_name                                                          mag
version                                                                  2
url                      http://snap.stanford.edu/ogb/data/nodeproppred...
add_inverse_edge                                                     False
has_node_attr                                                         True
has_edge_attr                                                        False
split                                                                 time
additional node files                                       

In [5]:
dataset1 = NodePropPredDataset(name = 'ogbn-mag')
dataset1

Loading necessary files...
This might take a while.


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]

Processing graphs...
Saving...


NodePropPredDataset(1)

In [6]:
features_mag = dataset1.graph['node_feat_dict']
features_mag

{'paper': array([[-0.095379,  0.040758, -0.210948, ...,  0.061569, -0.027663,
         -0.133832],
        [-0.151047, -0.107315, -0.221964, ...,  0.345754, -0.027737,
         -0.218527],
        [-0.114799, -0.175982, -0.260556, ...,  0.173058, -0.156445,
         -0.277954],
        ...,
        [ 0.022815, -0.0865  ,  0.098138, ..., -0.054667, -0.207721,
         -0.230458],
        [-0.289148, -0.202898, -0.152454, ...,  0.104207,  0.204123,
         -0.352805],
        [-0.088966, -0.034788, -0.264226, ...,  0.260077, -0.087453,
         -0.517127]], dtype=float32)}

In [7]:
print(dataset1.graph['num_nodes_dict'])

{'author': 1134649, 'field_of_study': 59965, 'institution': 8740, 'paper': 736389}


In [8]:
print(features_mag['paper'].shape)
print(type(features_mag['paper']))

(736389, 128)
<class 'numpy.ndarray'>


In [9]:
type(graph)

dgl.heterograph.DGLHeteroGraph

In [10]:
graph.ndata['features']={'paper':torch.from_numpy(features_mag['paper'])}

In [11]:
graph

Graph(num_nodes={'author': 1134649, 'field_of_study': 59965, 'institution': 8740, 'paper': 736389},
      num_edges={('author', 'affiliated_with', 'institution'): 1043998, ('author', 'writes', 'paper'): 7145660, ('paper', 'cites', 'paper'): 5416271, ('paper', 'has_topic', 'field_of_study'): 7505078},
      metagraph=[('author', 'institution', 'affiliated_with'), ('author', 'paper', 'writes'), ('paper', 'paper', 'cites'), ('paper', 'field_of_study', 'has_topic')])

In [12]:
graph.ndata['features']['paper']

tensor([[-0.0954,  0.0408, -0.2109,  ...,  0.0616, -0.0277, -0.1338],
        [-0.1510, -0.1073, -0.2220,  ...,  0.3458, -0.0277, -0.2185],
        [-0.1148, -0.1760, -0.2606,  ...,  0.1731, -0.1564, -0.2780],
        ...,
        [ 0.0228, -0.0865,  0.0981,  ..., -0.0547, -0.2077, -0.2305],
        [-0.2891, -0.2029, -0.1525,  ...,  0.1042,  0.2041, -0.3528],
        [-0.0890, -0.0348, -0.2642,  ...,  0.2601, -0.0875, -0.5171]])

In [13]:
label

{'paper': tensor([[246],
         [131],
         [189],
         ...,
         [266],
         [289],
         [  1]])}

In [16]:
from dgl.data.utils import save_graphs

save_graphs('./ogbm-mag.bin',graph,label)

In [15]:
def create_mask_from_idx(idx,total_nodes):
    mask_array = np.zeros(total_nodes,dtype=bool)
    mask_array[idx]=True
    return mask_array